In [1]:
from pathlib import Path
from argparse import Namespace
import matplotlib.pyplot as plt
import av2.rendering.vector as vector_plotting_utils
from shapely.geometry import Polygon

from MapGraph import MapGraph
from ActorGraph import ActorGraph

from av2.map.map_api import ArgoverseStaticMap
from av2.datasets.motion_forecasting import scenario_serialization

from av2.datasets.motion_forecasting.viz.scenario_visualization import (
    visualize_scenario
)
from av2.map.map_api import ArgoverseStaticMap

from graph_creator.create_graph import get_scenario_data, plot_scene_at_timestep

ImportError: cannot import name 'plot_scene_at_timestep' from 'av2.datasets.motion_forecasting.viz.scenario_visualization' (/Users/marius/code/graph_coverage/.venv/lib/python3.13/site-packages/av2/datasets/motion_forecasting/viz/scenario_visualization.py)

In [2]:
repo_root = Path( ".").resolve().parents[0]
dataroot = repo_root /  "argoverse_data" / "train"
print(repo_root)


/Users/marius/code/graph_coverage


In [3]:
scenario_folders = [f for f in dataroot.iterdir() if f.is_dir()]

# Print number of scenarios found
print(f"Found {len(scenario_folders)} scenarios")

Found 199908 scenarios


In [ ]:
# Only process first 5 scenarios
actor_graphs = []

for i, scenario in enumerate(scenario_folders):
    if i >= 50:
        break
    log_id = scenario.name
    scenario, map = get_scenario_data(dataroot, log_id)
    G_map = MapGraph.create_from_argoverse_map(map)
    actor_graph = ActorGraph.from_argoverse_scenario(
                                        scenario, 
                                        G_map, 
                                        delta_timestep_s=100.0,
                                        max_distance_lead_veh_m=200,
                                        max_distance_opposite_veh_m=400,
                                        max_distance_neighbor_forward_m=200,
                                        max_distance_neighbor_backward_m=100,
    )
    actor_graphs.append(actor_graph)
    actor_graph.visualize_actor_graph(t_idx=0.0, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))  
    visualize_scenario(scenario, map, save_path=(repo_root / "plots" / f"{log_id}_video_.mp4") )
    timestamp=0.0
    plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=False)

In [5]:
from graph_creator.SubgraphExtractor import SubgraphExtractor
import os

# Example usage of the SubgraphExtractor
def analyze_actor_graphs(actor_graphs, strategy='frequency', output_dir=None):
    """
    Analyze actor graphs to find common subgraphs using the specified strategy.
    
    Args:
        actor_graphs: List of ActorGraph objects
        strategy: Strategy for selecting subgraphs
        output_dir: Directory to save visualizations
    
    Returns:
        SubgraphExtractor instance
    """
    print(f"Analyzing actor graphs using '{strategy}' strategy...")
    
    # Initialize extractor with specified strategy
    extractor = SubgraphExtractor(
        subgraph_selection_strategy=strategy,
        min_subgraph_size=2,
        max_subgraph_size=None,  # No limit
        # Custom matchers can be defined here if needed
    )
    
    # Extract subgraphs
    subgraph_library = extractor.extract_subgraphs(actor_graphs)
    
    # Print properties
    extractor.print_subgraph_properties()
    
    # Visualize subgraphs
    if output_dir is not None:
        strategy_dir = os.path.join(output_dir, f"strategy_{strategy}")
        extractor.visualize_subgraphs(output_dir=strategy_dir)
    else:
        extractor.visualize_subgraphs()
    
    return extractor

# Create visualization output directory
output_dir = repo_root /  "plots" / "subgraph_analysis"
os.makedirs(output_dir, exist_ok=True)

# Analyze with different strategies
strategies = ['frequency', 'density', 'coverage', 'complexity', 'size']
extractors = {}

for strategy in strategies:
    extractors[strategy] = analyze_actor_graphs(actor_graphs, strategy, output_dir)

# Example of how to access the subgraphs for a specific actor graph and timestamp
print("\nExample of subgraph decomposition:")
print("=================================")
example_graph_idx = 0  # First graph in the list
example_timestamp = list(actor_graphs[example_graph_idx].actor_graphs.keys())[0]
print(f"Graph {example_graph_idx}, Timestamp {example_timestamp}:")

for sg_info in actor_graphs[example_graph_idx].actor_subgraphs[example_timestamp]:
    sg_id = sg_info['subgraph_id']
    nodes = sg_info['nodes']
    print(f"  Subgraph ID: {sg_id}, Nodes: {nodes}")
    # Additional information can be accessed from sg_info['node_mapping']

Analyzing actor graphs using 'frequency' strategy...

Subgraph Library:
Subgraph ID: 2
  Frequency: 30
  Nodes: 2
  Edges: 2
  Edge types:
    - following_lead: 1
    - leading_vehicle: 1

Subgraph ID: 10
  Frequency: 16
  Nodes: 3
  Edges: 4
  Edge types:
    - neighbor_vehicle: 4

Subgraph ID: 11
  Frequency: 9
  Nodes: 3
  Edges: 6
  Edge types:
    - neighbor_vehicle: 4
    - following_lead: 1
    - leading_vehicle: 1

Subgraph ID: 9
  Frequency: 5
  Nodes: 4
  Edges: 10
  Edge types:
    - neighbor_vehicle: 4
    - opposite_vehicle: 4
    - following_lead: 1
    - leading_vehicle: 1

Subgraph ID: 19
  Frequency: 3
  Nodes: 4
  Edges: 6
  Edge types:
    - neighbor_vehicle: 4
    - following_lead: 1
    - leading_vehicle: 1

Subgraph ID: 22
  Frequency: 3
  Nodes: 4
  Edges: 6
  Edge types:
    - neighbor_vehicle: 6

Subgraph ID: 1
  Frequency: 1
  Nodes: 4
  Edges: 8
  Edge types:
    - opposite_vehicle: 6
    - following_lead: 1
    - leading_vehicle: 1

Subgraph ID: 3
  Frequenc